In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [6]:
cities, minimum_require = gp.multidict({
    'Airdie' : 52000,
    'Redeer': 96000,
    'Olds': 8000,
    'Carstair' : 4000
})

company, supply_available,cost = gp.multidict({
    'Epcor' : [90000, 12.01],
    'Enamax' : [70000, 12.06],
    'Direct_energy': [60000, 12.02]
})

moving_supplies = []
days = {1,2}
for i in company:
    for j in cities:
        moving_supply = (i,j)
        moving_supplies.append(moving_supply)


moving_supplies2 = []
supplies_holder = []
for i in company:
    for j in cities:
        for k in days:
            moving_supply2 = (i,j,k)
            if k < max(days):
                supplies_holder.append(moving_supply2)
            
            moving_supplies2.append(moving_supply2)
            



In [7]:
# second day demand increase by 10% and cost increase by 20%

# cost for sencond day
cost2 = {i : int for i in company}
for i in company:
    cost2[i] = cost[i] +(cost[i] *0.2)

# minimum require for each cities on secnond day
minimum_require2 = {i : str for i in cities}
for i in cities:
    minimum_require2[i] = minimum_require[i] + (minimum_require[i] * 0.1)

In [ ]:
model = gp.Model(name = 'flow_cost')
electric_supply = model.addVars(moving_supplies, name = 'moving_supplies')
electric_supply_second = model.addVars(moving_supplies, name = 'moving_supplies2' )

model.addConstr(electric_supply['Epcor', 'Airdie'] == 0, name = 'unsupported1')
model.addConstr(electric_supply['Enamax', 'Redeer'] == 0, name = 'unsupported2')
model.addConstr(electric_supply_second['Epcor', 'Airdie'] == 0, name = 'unsupported1')
model.addConstr(electric_supply_second['Enamax', 'Redeer'] == 0, name = 'unsupported2')

# first month
city_vars = {city: [] for city in cities}
supplier_vars = {supp: [] for supp in company}

for i, j in electric_supply.keys():
    city_vars[j].append(electric_supply[i, j])
    supplier_vars[i].append(electric_supply[i, j])

# Demand constraints: each city must receive exactly its required amount
for city in cities:
    model.addConstr(gp.quicksum(city_vars[city]) == minimum_require[city], 
                    name=f'demand_{city}')

# Supply constraints: each supplier can supply at most its available amount
for supp in company:
    model.addConstr(gp.quicksum(supplier_vars[supp]) <= supply_available[supp], 
                    name=f'supply_{supp}')
    
#second month 
city_vars2 = {city:[] for city in cities}
supplier_vars2 = {supp:[] for supp in company}

for i,j in electric_supply_second.keys():
    city_vars2[j].append(electric_supply_second[i,j])
    supplier_vars2[i].append(electric_supply_second[i,j])

for city in cities:
    model.addConstr(gp.quicksum(city_vars2[city]) == minimum_require2[city], name=f'demand_{city}')


for supp in company:
    model.addConstr(gp.quicksum(supplier_vars2[supp]) <= supply_available[supp], 
                    name=f'supply_{supp}')
    
# Objective function: minimize total cost
firstmonth_cost = gp.quicksum(electric_supply[i, j] * cost[i] for i, j in electric_supply.keys())
secondmonth_cost = gp.quicksum(electric_supply_second[i,j] * cost2[i] for i,j in electric_supply_second.keys())

total_cost = firstmonth_cost + secondmonth_cost
model.setObjective(total_cost, gp.GRB.MINIMIZE)

# Solve the model
model.optimize()
    

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 18 rows, 24 columns and 52 nonzeros
Model fingerprint: 0xf34bf666
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+03, 1e+05]
Presolve removed 12 rows and 14 columns
Presolve time: 0.03s
Presolved: 6 rows, 10 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.4604563e+06   2.905550e+03   0.000000e+00      0s
       5    4.4614920e+06   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds (0.00 work units)
Optimal objective  4.461492000e+06


In [16]:
day2=[]
for i, j in electric_supply_second.keys():
    day2.append(electric_supply_second[i,j].X * cost2[i])

print(gp.quicksum(day2))

2538792.0


In [9]:
model2 = gp.Model(name = 'model2days')
# supply in the day
day_supplies = model2.addVars(moving_supplies2, name = 'day_supplies')
# supply purchases for the next days
hold_supplies = supplier_store = model2.addVars(supplies_holder, name ='hold_supply')

# create variable holder for city and oppereator based on the day
city_vars_day ={}
suply_vars_day = {}
supply_vars_hold ={}
for city in cities:
    for day in days:
        city_vars_day[city, day] =[]

for supp in company:
    for day in days:
        suply_vars_day[supp, day] =[]

# add the design parameter for each holder
for i,j,k in day_supplies.keys():
    city_vars_day[j,k].append(day_supplies[i,j,k])
    suply_vars_day[i,k].append(day_supplies[i,j,k])

# setting unspported area for 2 days
for i in days:
    model2.addConstr(day_supplies['Epcor', 'Airdie', i] == 0, name = 'unsupported1')
    model2.addConstr(day_supplies['Enamax', 'Redeer',i] == 0, name = 'unsupported2')
    model2.addConstr(
        gp.quicksum(hold_supplies) <= gp.quicksum(supply_available) - gp.quicksum(minimum_require)
        ,name='avalable_hold')

for i in range(1,len(days)):
    model2.addConstr(hold_supplies['Epcor', 'Airdie', i] == 0, name = 'unsupporth1')
    model2.addConstr(hold_supplies['Enamax', 'Redeer',i]  == 0, name = 'unsupporth2') 

# add the limitation for the requirement of cities on different day
for day in days:
    for city in cities:
        if day ==1 : # the demand on 2 days is different
            model2.addConstr(gp.quicksum(city_vars_day[city,day]) == minimum_require[city])
        else:
            model2.addConstr(gp.quicksum(city_vars_day[city,day]) == minimum_require2[city])

# add the limitation of supply for each company
for day in days:
    for supp in company:# same limitation of supply for 2 days
        model2.addConstr(gp.quicksum(suply_vars_day[supp,day]) <= supply_available[supp])


for day in days:
    if day ==1:
        day_cost1 = gp.quicksum(day_supplies[i,j,k] * cost[i] for i,j,k in day_supplies.keys())
        hold_cost =  gp.quicksum(hold_supplies[i,j,k] * cost[i] for i,j,k in hold_supplies.keys())
    if day ==2:
        day_cost2 = gp.quicksum(day_supplies[i,j,k] * cost2[i] for i,j,k in day_supplies.keys())

total_cost = day_cost1 + day_cost2 + hold_cost

model2.setObjective(total_cost, GRB.MINIMIZE)

model2.optimize()


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 22 rows, 36 columns and 78 nonzeros
Model fingerprint: 0x1d84fe03
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+03, 1e+05]
Presolve removed 16 rows and 26 columns
Presolve time: 0.02s
Presolved: 6 rows, 10 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.8823465e+06   2.905550e+03   0.000000e+00      0s
       5    8.8843920e+06   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.10 seconds (0.00 work units)
Optimal objective  8.884392000e+06


/var/folders/v8/wksvxw9d4pddcx_9trfddjsw0000gn/T/ipykernel_1762/4159591427.py:29: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  gp.quicksum(hold_supplies) <= gp.quicksum(supply_available) - gp.quicksum(minimum_require)


In [10]:
print(hold_supplies)


{('Epcor', 'Airdie', 1): <gurobi.Var hold_supply[Epcor,Airdie,1] (value 0.0)>, ('Epcor', 'Redeer', 1): <gurobi.Var hold_supply[Epcor,Redeer,1] (value 0.0)>, ('Epcor', 'Olds', 1): <gurobi.Var hold_supply[Epcor,Olds,1] (value 0.0)>, ('Epcor', 'Carstair', 1): <gurobi.Var hold_supply[Epcor,Carstair,1] (value 0.0)>, ('Enamax', 'Airdie', 1): <gurobi.Var hold_supply[Enamax,Airdie,1] (value 0.0)>, ('Enamax', 'Redeer', 1): <gurobi.Var hold_supply[Enamax,Redeer,1] (value 0.0)>, ('Enamax', 'Olds', 1): <gurobi.Var hold_supply[Enamax,Olds,1] (value 0.0)>, ('Enamax', 'Carstair', 1): <gurobi.Var hold_supply[Enamax,Carstair,1] (value 0.0)>, ('Direct_energy', 'Airdie', 1): <gurobi.Var hold_supply[Direct_energy,Airdie,1] (value 0.0)>, ('Direct_energy', 'Redeer', 1): <gurobi.Var hold_supply[Direct_energy,Redeer,1] (value 0.0)>, ('Direct_energy', 'Olds', 1): <gurobi.Var hold_supply[Direct_energy,Olds,1] (value 0.0)>, ('Direct_energy', 'Carstair', 1): <gurobi.Var hold_supply[Direct_energy,Carstair,1] (valu